<a href="https://colab.research.google.com/github/Purplegay/Cornchicken/blob/master/Image_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!apt-get -y install -qq aria2
!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
!pip install -q --pre triton

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd /content/stable-diffusion-webui/extensions/

# tag自动补全插件（可选）
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git tag-autocomplete

# 下载审美渐变插件（可选）
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-aesthetic-gradients aesthetic-gradients
    
# 下载历史记录/图像浏览器插件（可选）
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser images-browser
    
# 中文UI界面翻译插件（可选）
!git clone https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN localization-zh_CN
    
# lora networks插件（可选）
!git clone https://github.com/kohya-ss/sd-webui-additional-networks sd-webui-additional-networks

#controlnet插件（可选）
!git clone https://jihulab.com/hunter0725/sd-webui-controlnet sd-webui-controlnet




In [ ]:
%cd /content/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\[t\]/self.logvar\[t.item()\]/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i '$a fastapi==0.90.0' requirements_versions.txt


In [ ]:
!python -m pip install --upgrade fastapi==0.90.1 

In [ ]:
!wget https://civitai.com/api/download/models/11745 -O /content/stable-diffusion-webui/models/Stable-diffusion/Chilloutmix.safetensors

In [ ]:
!wget https://civitai.com/api/download/models/8750 -O /content/stable-diffusion-webui/models/Lora/koreanDollLikeness_v10.safetensors
!wget https://civitai.com/api/download/models/12050 -O /content/stable-diffusion-webui/models/Lora/japaneseDollLikeness_v10.safetensors

In [ ]:
!wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -O /content/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors

In [ ]:

%cd /content/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --gradio-debug --xformers" REQS_FILE="requirements.txt" python launch.py

 27% 4/15 [00:08<00:22,  2.04s/it]
 33% 5/15 [00:10<00:20,  2.05s/it]
 40% 6/15 [00:12<00:18,  2.06s/it]
 47% 7/15 [00:14<00:16,  2.06s/it]
 53% 8/15 [00:16<00:14,  2.07s/it]
 60% 9/15 [00:18<00:12,  2.08s/it]
 67% 10/15 [00:20<00:10,  2.09s/it]
 73% 11/15 [00:22<00:08,  2.11s/it]
 80% 12/15 [00:24<00:06,  2.12s/it]
 87% 13/15 [00:27<00:04,  2.13s/it]
 93% 14/15 [00:29<00:02,  2.13s/it]
100% 15/15 [00:30<00:00,  2.02s/it]
Total progress: 100% 15/15 [00:30<00:00,  2.03s/it]
